In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np


In [2]:
# gather data
boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['AGE', 'INDUS'], axis=1)
# features.head()
log_prices = np.log(boston_dataset.target)
# log_prices.shape
target = pd.DataFrame(log_prices, columns=['PRICES'])

In [3]:
CRIM_IDX=0
ZN_IDX=1
CHAS_IDX=2
NOX_IDX=3
RM_IDX = 4
PTRATIO_IDX = 8
# property_stats = np.ndarray(shape=(1,11))
# property_stats[0][CRIM_IDX] = features['CRIM'].mean()
# property_stats[0][ZN_IDX] = features['ZN'].mean()
# property_stats[0][CHAS_IDX] = features['CHAS'].mean()

property_stats = features.mean().values.reshape(1,11)
property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [4]:
print(type(features.mean()))
print(type(features.mean().values))
print(type(features.mean().values.shape))

<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'tuple'>


In [5]:
# features.mean().values.reshape(1,11)

In [6]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

# calculate MSE and RMSE
MSE = mean_squared_error(target , fitted_vals)
RMSE = np.sqrt(MSE)

In [7]:
def get_log_estimate(nr_room,
                    stud_per_class,
                    next_to_river = False,
                    high_confidence=True):
#     config property
    property_stats[0][RM_IDX] = nr_room
    property_stats[0][PTRATIO_IDX] = stud_per_class

    if next_to_river:
            property_stats[0][CHAS_IDX] = 1
    else:
            property_stats[0][CHAS_IDX] = 0

    # make prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    #cal range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68

    return log_estimate,upper_bound, lower_bound, interval



get_log_estimate(3,19)

(2.709035644911383, 3.0840599153056436, 2.3340113745171225, 95)

In [11]:
# features.head()
np.median(boston_dataset.target)

21.2

In [14]:
# Writing a python code for estimating the 1970s price using log price estimate
# As well as Uppper and lower bound for todays price
# nearest 1000s Dollars

ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

log_est, upper, lower, conf = get_log_estimate(9, 15,False, False)

# Convert to today's dollars
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR

# Round the dollar values to nearest thousand
rounded_est = np.around(dollar_est, -3)
rounded_hi = np.around(dollar_hi, -3)
rounded_low = np.around(dollar_low, -3)

print(f'The estimated property value is {rounded_est}.')
print(f'At {conf}% confidence the valuation range is')
print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

The estimated property value is 827000.0.
At 68% confidence the valuation range is
USD 685000.0 at the lower end to USD 997000.0 at the high end.


In [31]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    """Estimate the price of a property in Boston.
    
    Keyword arguments:
    rm -- number of rooms in the property.
    ptratio -- number of students per teacher in the classroom for the school in the area.
    chas -- True if the property is next to the river, False otherwise.
    large_range -- True for a 95% prediction interval, False for a 68% interval.
    
    """
                
    
    if rm < 1 or ptratio < 1:
        print('That is unrealistic. Try again.')
        return

    log_est, upper, lower, conf = get_log_estimate(rm, 
                                                   stud_per_class=ptratio, 
                                                   next_to_river=chas, 
                                                   high_confidence=large_range)

    # Convert to today's dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # Round the dollar values to nearest thousand
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'The estimated property value is {rounded_est}.')
    print(f'At {conf}% confidence the valuation range is')
    print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

In [39]:
get_dollar_estimate(rm=5, ptratio=0, chas=True)

That is unrealistic. Try again.


In [35]:
# get_dollar_estimate(6, 12, False)
get_dollar_estimate(6, 12, True)

The estimated property value is 783000.0.
At 95% confidence the valuation range is
USD 538000.0 at the lower end to USD 1139000.0 at the high end.
